In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import math
import random

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from scipy.spatial.transform import Rotation
from tqdm import tqdm,trange

from numpy.linalg import inv

import imgaug.augmenters as iaa

from lib.utils.tracker_new import Sort as Tracker

In [2]:
def read_calib(path):
    with open(path, 'r') as f:
        lines = f.readlines()
        P2 = np.array(lines[2].strip().split(' ')[1:], dtype=np.float32).reshape(3, 4)
        
    return P2

def get_corners(data,rx=0,ry=0,rz=0,x=None,y=None,z=None):
    
    x = data['x'] if x == None else x
    y = data['y'] if y == None else y
    z = data['z'] if z == None else z

    R = Rotation.from_euler('zxy', [rz,rx,ry], degrees=False).as_matrix()
    
#     R = roty(data['yaw'])
    l = float(data['l'])
    w = float(data['w'])
    h = float(data['h'])
    
    x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2]
#     y_corners = [0,0,0,0,h,h,h,h]
    y_corners = [h/2,h/2,h/2,h/2,-h/2,-h/2,-h/2,-h/2]
    z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2]
    
    corners = np.dot(R,np.vstack([x_corners, y_corners, z_corners]))


#     translate from origin 
    corners[0,:] = corners[0,:] + x
    corners[1,:] = corners[1,:] + y
    corners[2,:] = corners[2,:] + z
#     corners = np.dot(R,corners)
    return corners

def project_3d(P,corner):
    conn = np.concatenate((corner.T, np.ones((8, 1))), axis=1)
    corners_img_before = np.matmul(conn, P.T)
    corners_img = corners_img_before[:, :2] / corners_img_before[:, 2][:, None]
    
    return corners_img

def random_color():
    r = random.randint(0, 255)
#     g = random.randint(0, 255)
#     b = random.randint(0, 255)
    rand_color = (255, 0, 0)
    return rand_color

def plot_img(plot,pts):
    
    color = random_color()
    # TOP SIDE EFGH
    plot = draw_line(plot,pts[4],pts[5],color=color)
    plot = draw_line(plot,pts[5],pts[6],color=color)
    plot = draw_line(plot,pts[6],pts[7],color=color)
    plot = draw_line(plot,pts[7],pts[4],color=color)

    # BOT SIDE ABCD
#     plot = draw_line(plot,pts[0],pts[1],color=(255,0,0))
#     plot = draw_line(plot,pts[1],pts[2],color=(255,0,0))
#     plot = draw_line(plot,pts[2],pts[3],color=(255,0,0))
#     plot = draw_line(plot,pts[3],pts[0],color=(255,0,0))
    
    plot = draw_line(plot,pts[0],pts[1],color=color)
    plot = draw_line(plot,pts[1],pts[2],color=color)
    plot = draw_line(plot,pts[2],pts[3],color=color)
    plot = draw_line(plot,pts[3],pts[0],color=color)

    # TIANG AE BF CG DH
    plot = draw_line(plot,pts[0],pts[4],color=color)
    plot = draw_line(plot,pts[1],pts[5],color=color)
    plot = draw_line(plot,pts[2],pts[6],color=color)
    plot = draw_line(plot,pts[3],pts[7],color=color)
    
#     plot = draw_line(plot,pts[0],pts[5],color=(255,0,0))
#     plot = draw_line(plot,pts[4],pts[1],color=(255,0,0))
    return plot

def draw_line(img,ptA,ptB,color=(255, 0, 0)):
    start_point = (int(ptA[0]),int(ptA[1]))
    end_point = (int(ptB[0]),int(ptB[1]))
    thickness = 1
    
    img = cv2.line(img, start_point, end_point, color, thickness)
    return img

def draw_rect(img,data,color=(0, 255, 0)):
    x0,y0 = int(data['xmin']),int(data['ymin'])
    x1,y1 = int(data['xmax']),int(data['ymax'])
    
    img = cv2.rectangle(img,(x0,y0),(x1,y1), color, 2)
    return img

def roty(t):
    c = np.cos(t)
    s = np.sin(t)
    return np.array([[c, 0, s], 
                     [0, 1, 0], 
                     [-s, 0, c]])

def calc_theta_ray(width, xmin,xmax, proj_matrix,is_y=False):
#     if is_y:
#         temp = xmin
#         xmin = 512-xmax
#         xmax = 512-temp
    
    fovx = 2 * np.arctan(width / (2 * proj_matrix[0][0]))
    center = xmin + abs(xmax-xmin)/2
    
    center = width - center if is_y else center
    
    dx = center - (width / 2)

    mult = 1
    if dx < 0:
        mult = -1
    dx = abs(dx)
    angle = np.arctan( (2*dx*np.tan(fovx/2)) / width )
    angle = angle * mult

    angle = fovx/2 - angle
    return angle

def get_theta(width,xmin,xmax,P,is_y=False):
    center = xmin + abs(xmax-xmin)/2
    F = P[0][0]
    dx = center - (width/2)
    dx = width - dx if is_y else dx
    theta = np.arctan(dx/F)
    
    theta = theta + np.pi/2 if is_y else theta
    return theta

def calc_theta_ray(width, xmin,xmax, proj_matrix,is_y=False):
    
    fovx = 2 * np.arctan(width / (2 * proj_matrix[0][0]))
    center = xmin + abs(xmax-xmin)/2
    
    center = width - center if is_y else center
    
    dx = center - (width / 2)

    mult = 1
    if dx < 0:
        mult = -1
    dx = abs(dx)
    angle = np.arctan( (2*dx*np.tan(fovx/2)) / width )
    angle = angle * mult

    angle = fovx/2 - angle
    return angle

def calc_theta_ray_center(width, center, proj_matrix,is_y=False):
    fovx = 2 * np.arctan(width / (2 * proj_matrix[0][0]))
    
    center = width - center if is_y else center
    
    dx = center - (width / 2)

    mult = 1
    if dx < 0:
        mult = -1
    dx = abs(dx)
    angle = np.arctan( (2*dx*np.tan(fovx/2)) / width )
    angle = angle * mult

    angle = fovx/2 - angle
    return angle

def get_ry(img_size,t):
    return (calc_theta_ray_center(img_size,t['cx'],P) + t['alphax'])

def get_rx(img_size,t):
    return -(calc_theta_ray_center(img_size,t['cy'],P,is_y=True) + t['alphay']) * -1

def get_xy(ann,center,P,c=0):
    
    center = np.array(center).reshape((1,2))
    depth = np.array(ann['z']).reshape(1,1)

    return imagetocamera(center,depth,P)

def imagetocamera(points, depth, projection):
    """
    points: (N, 2), N points on X-Y image plane
    depths: (N,), N depth values for points
    projection: (3, 4), projection matrix
    corners: (N, 3), N points on X(right)-Y(down)-Z(front) camera coordinate
    """
    assert points.shape[1] == 2, "Shape ({}) not fit".format(points.shape)

    corners = np.hstack([points, np.ones(
        (points.shape[0], 1))]).dot(inv(projection[:, 0:3]).T)
    assert np.allclose(corners[:, 2], 1)
    corners *= depth.reshape(-1, 1)

    return list(corners[0])

In [3]:
def bbox_iou(boxA, boxB):
    # boxA and boxB are lists or tuples of (x1, y1, x2, y2)
    # representing the top-left corner coordinates (x1, y1)
    # and the bottom-right corner coordinates (x2, y2) of the boxes

    # determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    intersection_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxA_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the
    # intersection area and dividing it by the sum of prediction
    # + ground-truth areas - the intersection area
    iou = intersection_area / float(boxA_area + boxB_area - intersection_area)

    # return the intersection over union value
    return iou


In [4]:
BASE_DIR_calib = '/home/alfin/Documents/deep_learning/production/centernet_3d_fish/'
# BASE_DIR = BASE_DIR_calib
BASE_DIR = '/home/alfin/Documents/deep_learning/fish_conversion/data/20221121_centernet_rxry/KITTI/detection/training/sample_video/'

# IMG_DIR = os.path.join(BASE_DIR,'trainval/')
IMG_DIR = os.path.join(BASE_DIR,'output_3/')
INF_DIR = os.path.join(BASE_DIR,'label_inference/')
CALIB_DIR = os.path.join(BASE_DIR_calib,'data/')

data_path = [{'img':IMG_DIR+img,'inf':INF_DIR+inf
             } for img,inf in 
             zip(sorted(os.listdir(IMG_DIR)),
                 sorted([x for x in os.listdir(INF_DIR) if not x.endswith('track.txt')])
                )]

calib_path = os.path.join(CALIB_DIR,'000000.txt')

In [5]:
data_path

[{'img': '/home/alfin/Documents/deep_learning/fish_conversion/data/20221121_centernet_rxry/KITTI/detection/training/sample_video/output_3/000000.jpg',
  'inf': '/home/alfin/Documents/deep_learning/fish_conversion/data/20221121_centernet_rxry/KITTI/detection/training/sample_video/label_inference/000000.txt'},
 {'img': '/home/alfin/Documents/deep_learning/fish_conversion/data/20221121_centernet_rxry/KITTI/detection/training/sample_video/output_3/000001.jpg',
  'inf': '/home/alfin/Documents/deep_learning/fish_conversion/data/20221121_centernet_rxry/KITTI/detection/training/sample_video/label_inference/000001.txt'},
 {'img': '/home/alfin/Documents/deep_learning/fish_conversion/data/20221121_centernet_rxry/KITTI/detection/training/sample_video/output_3/000002.jpg',
  'inf': '/home/alfin/Documents/deep_learning/fish_conversion/data/20221121_centernet_rxry/KITTI/detection/training/sample_video/label_inference/000002.txt'},
 {'img': '/home/alfin/Documents/deep_learning/fish_conversion/data/202

In [6]:
len(data_path)

300

In [7]:
img_input_size = (512,512)

tracker = Tracker()
inf_header = ['h','w','l','z','alphax','alphay','conf','idx','cx','cy','xmin','ymin','xmax','ymax','id']

for j,data in tqdm(enumerate(data_path[:])):
#     try:
    tracking_result = list()
    ann = pd.read_csv(data['inf'],sep = ' ',names=inf_header)

    P = read_calib(calib_path)
    detections = []
    
    for i in range(len(ann)):
        label = ann.iloc[i]

        label_dict = label.to_dict()
        label_dict['id'] = 0
        
        xmin = label['xmin']
        ymin = label['ymin']
        xmax = label['xmax']
        ymax = label['ymax']
        
#         center = (int(label['cx']),int(label['cy']))
#         x,y,z = get_xy(label,center,P)

#         ry = get_ry(img_input_size[0],label)
#         rx = get_rx(img_input_size[1],label)
#         rz = 0

#         cor = get_corners(label,rx,ry,rz,x=x,y=y,z=z)

#         pts = project_3d(P,cor)

#         xs = [p[0] for p in pts]
#         ys = [p[1] for p in pts]

#         xmin = int(min(xs))
#         ymin = int(min(ys))
#         xmax = int(max(xs))
#         ymax = int(max(ys))

        detections.append([xmin,ymin,xmax,ymax,label['conf'],label_dict])
    
    if len(detections)==0:
        continue
    detections_np = np.array([d[:5] for d in detections])
    track_ids = tracker.update(detections_np)
#     print(track_ids)
    for t in track_ids:
        box = [0,0,0,0, 0, None]
        
        for d in detections:
            if (bbox_iou(d[:5],t[:5])) > box[4]:
                box = d
                d[-1]['id'] = int(t[-1])

        tracking_result.append(box[-1])
    
#     print(tracking_result)
    df = pd.DataFrame(tracking_result)
    if len(df>0):
        df = df[['h','w','l','z','alphax','alphay','conf','idx','cx','cy','xmin','ymin','xmax','ymax','id']]
    save_path = os.path.splitext(data['inf'])[0]+'_track.txt'
    df.to_csv(save_path,header=False,sep=' ',index=False)

300it [00:01, 158.77it/s]


In [8]:
df

,h,w,l,z,alphax,alphay,conf,idx,cx,cy,xmin,ymin,xmax,ymax,id
0,0.133950,0.061091,0.307553,1.787999,5.283892,5.579156,0.956417,299.0,164.0,237.0,137.0,225.0,189.0,247.0,804
1,0.144823,0.065895,0.332498,1.157228,0.214040,5.744151,0.991570,299.0,224.0,143.0,200.0,103.0,252.0,172.0,747
2,0.154706,0.070251,0.355632,2.221949,5.761298,5.826317,0.844476,299.0,164.0,160.0,144.0,144.0,184.0,174.0,802
3,0.158302,0.072108,0.363932,1.713391,2.073165,5.514184,0.714544,299.0,253.0,376.0,221.0,358.0,284.0,395.0,798
4,0.144823,0.065895,0.332498,1.157228,0.214040,5.744151,0.991570,299.0,224.0,143.0,200.0,103.0,252.0,172.0,747
5,0.151973,0.069223,0.348819,1.093937,1.623867,5.252581,0.973411,299.0,357.0,267.0,335.0,244.0,374.0,293.0,679
6,0.153597,0.069972,0.352853,1.415655,1.662991,4.729012,0.964326,299.0,320.0,339.0,291.0,316.0,344.0,366.0,710
7,0.151973,0.069223,0.348819,1.093937,1.623867,5.252581,0.973411,299.0,357.0,267.0,335.0,244.0,374.0,293.0,679
8,0.149350,0.067838,0.342891,1.566285,1.848637,4.600142,0.919595,299.0,324.0,316.0,294.0,299.0,351.0,335.0,710
9,0.151973,0.069223,0.348819,1.093937,1.623867,5.252581,0.973411,299.0,357.0,267.0,335.0,244.0,374.0,293.0,679


In [9]:
tracking_result

[{'h': 0.13395005,
  'w': 0.061091475,
  'l': 0.30755287,
  'z': 1.7879989,
  'alphax': 5.283892,
  'alphay': 5.5791564,
  'conf': 0.95641655,
  'idx': 299.0,
  'cx': 164.0,
  'cy': 237.0,
  'xmin': 137.0,
  'ymin': 225.0,
  'xmax': 189.0,
  'ymax': 247.0,
  'id': 804},
 {'h': 0.14482263,
  'w': 0.06589491,
  'l': 0.3324985,
  'z': 1.1572275,
  'alphax': 0.21403961,
  'alphay': 5.744151,
  'conf': 0.9915697,
  'idx': 299.0,
  'cx': 224.0,
  'cy': 143.0,
  'xmin': 200.0,
  'ymin': 103.0,
  'xmax': 252.0,
  'ymax': 172.0,
  'id': 747},
 {'h': 0.15470637,
  'w': 0.07025142,
  'l': 0.35563198,
  'z': 2.221949,
  'alphax': 5.7612977,
  'alphay': 5.826317,
  'conf': 0.84447557,
  'idx': 299.0,
  'cx': 164.0,
  'cy': 160.0,
  'xmin': 144.0,
  'ymin': 144.0,
  'xmax': 184.0,
  'ymax': 174.0,
  'id': 802},
 {'h': 0.15830201,
  'w': 0.07210832,
  'l': 0.36393234,
  'z': 1.7133911,
  'alphax': 2.0731647,
  'alphay': 5.514184,
  'conf': 0.71454436,
  'idx': 299.0,
  'cx': 253.0,
  'cy': 376.0,
  '